In [1]:
#set project path from config.yaml
import pathlib
import yaml

with open("config.yaml", "r") as f:
   config = yaml.safe_load(f)

project_root = pathlib.Path(config["project"]["root_path"])

print("Current project path: ", project_root)

Current project path:  C:\Users\Konstantinos\Desktop\Coughvid Data


In [2]:
import requests
#naming backup to keep the original dset zipped as a backup to avoid redownloading
zipdl_dir = project_root / "dsetv3_backup"
#automatically make path, avoiding having to create the files manually
zipdl_dir.mkdir(parents=True, exist_ok=True) 

#NOTE anth: to remove any confusion with _dir and _path suffixes
    #dir: goes to folder
    #path: goes to file
        #it's my way of separating what is what, hopefully it's not confusing for any future readers

#from gdrive link, *DO NOT CHANGE*

import gdown

file_id = "1XSY-5SFv_F2GEP9VZOTrPxE72N8aXYd6"
url = f"https://drive.google.com/uc?id={file_id}"
zip_path = zipdl_dir / "dsetv3.zip"

gdown.download(url, str(zip_path), quiet=False)




Downloading...
From (original): https://drive.google.com/uc?id=1XSY-5SFv_F2GEP9VZOTrPxE72N8aXYd6
From (redirected): https://drive.google.com/uc?id=1XSY-5SFv_F2GEP9VZOTrPxE72N8aXYd6&confirm=t&uuid=b05493dc-fbdd-47d4-adb1-785b075194e0
To: C:\Users\Konstantinos\Desktop\Coughvid Data\dsetv3_backup\dsetv3.zip
100%|██████████| 2.30G/2.30G [04:15<00:00, 8.98MB/s]


'C:\\Users\\Konstantinos\\Desktop\\Coughvid Data\\dsetv3_backup\\dsetv3.zip'

In [5]:
#unzip the dset in a separate file 
    #NOTE unzipping will result in a single file with both .webm (audio) and label (.json) files dumped together
    #will sort in separate directories to make life easier 

zip_path = zipdl_dir / "dsetv3.zip"

import zipfile
import os

dset_extracted_dir = project_root / "unzipped_dset" #dump the entire zip here 
labels_dir = project_root / "labels_json"
webm_dir = project_root / "audio_webm"
ogg_dir = project_root / "audio_ogg"
wav_dir = project_root / "audio_wav"

#create directories
dset_extracted_dir.mkdir(parents=True, exist_ok=True) 
labels_dir.mkdir(parents=True, exist_ok=True) 
webm_dir.mkdir(parents=True, exist_ok=True) 
ogg_dir.mkdir(parents = True, exist_ok= True)
wav_dir.mkdir(parents=True, exist_ok=True)


print(zip_path)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(dset_extracted_dir)

C:\Users\Konstantinos\Desktop\Coughvid Data\dsetv3_backup\dsetv3.zip


In [6]:
import shutil
from tqdm import tqdm

#reecursively get all files in the extracted dataset directory
    #there will only be one folder in dset_extracted_dir 
        #just doing it this way because the final folder name can have some variability due to how gdown names files 
all_files = [f for f in dset_extracted_dir.rglob("*") if f.is_file()]

for file_path in tqdm(all_files, desc="Sorting files"):
    suffix = file_path.suffix.lower()
    if suffix == ".json":
        shutil.move(str(file_path), labels_dir / file_path.name)
    elif suffix == ".webm":
        shutil.move(str(file_path), webm_dir / file_path.name)
    elif suffix == ".ogg":
        shutil.move(str(file_path), ogg_dir / file_path.name)
    elif suffix == ".wav":
        shutil.move(str(file_path), wav_dir / file_path.name)
    else:
        print(f"Skipping unknown file type: {file_path.name}")


Sorting files: 100%|██████████| 68869/68869 [00:35<00:00, 1914.13it/s]

Skipping unknown file type: metadata_compiled.csv
